# Ansatzes

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS(lnnize =True)

## 1. ansatzes module

One mandatory step for using the Parent Hamiltonian, **PH**, library (see nootebook **02_Using_PH_Class.ipynb**) is computing for a given ansatz its complete state. This is the amplitudes of the state in the computational *n* qubit basis. 

In the *ansatzes* module of the **PH** library several functions and classes for deling with this part of the computation was done.


### 1.1 SolveCircuit class

The **SolveCircuit** takes a *Atos myqlm circuit* with an ansatz, fix their parameter and simulates using *Atos qpu* and return the state of the ansatz. This class needs for initialization following arguments:

* qlm_circuit: *Atos myqlm circuit* with an ansatz
* kwargs: keyword arguments where following keys are proccesed:
    * parameters: It could be list or a dictionary with the parameters for fixing it the parameters of the ansatz or a None. In this case random parameters will be used
    * qpu: Atos qpu for simulate the ansatz
    
For showing how this class works some example circuits will be provided

In [ ]:
from PH.ansatzes import SolveCircuit

### 1.2 Parent Hamiltonian Github ansatz

The *ansatz_qlm_01* functions implements a **Atos myqlm** version of the ansatz in the github:

https://github.com/FumiKobayashi/Parent_Hamiltonian_as_a_benchmark_problem_for_variational_quantum_eigensolvers

from the original Parent Hamiltonian Papper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)

We need functions *ansatz_qlm_01* and the function *solve_ansatz* that allows the simulation of a **Atos myqlm** program

In [ ]:
from PH.ansatzes import ansatz_qlm_01

We need to provided to the *ansatz_qlm_01* the number of desired qubits and the depth of the circuit. The function returns an *Atos myqlm circuit* of the ansatz.

In [ ]:
n_qubits = 12
depth = 3
circ_ansatz01 = ansatz_qlm_01(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz01 --svg

Now we can initialize the *SolveCircuit* by providing the circuit and a input dictionary

In [ ]:
par_names = circ_ansatz01.get_variables()

In [ ]:
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : None,
}

In [ ]:
solv_ansatz01 = SolveCircuit(circ_ansatz01, **class_dict)

In [ ]:
# If None is provided then random parameters will be used
solv_ansatz01.parameters

In [ ]:
angle_list = [np.pi /2.0 for i in circ_ansatz01.get_variables()]
print("List with parameter will be provided: {}".format(angle_list))
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : angle_list,
}
solv_ansatz01 = SolveCircuit(circ_ansatz01, **class_dict)
solv_ansatz01.parameters

In [ ]:
angle_list = {v: np.pi /4.0 for v in circ_ansatz01.get_variables()}
print("Dictionary with parameter will be provided: {}".format(angle_list))
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : angle_list,
}
solv_ansatz01 = SolveCircuit(circ_ansatz01, **class_dict)
solv_ansatz01.parameters

Additionally we can set the parameter attribute after initialization

In [ ]:
solv_ansatz01.parameters = None
solv_ansatz01.parameters

In [ ]:
solv_ansatz01.parameters = [np.pi /2.0 for i in solv_ansatz01.parameter_names]
solv_ansatz01.parameters

In [ ]:
solv_ansatz01.parameters = {i: np.pi /4.0 for i in solv_ansatz01.parameter_names}
solv_ansatz01.parameters

For simulating the ansatz the *run* method of the class should be executed when properly configuration is done. The *state* attribute will contain a pandas DataFrame with complet state information

In [ ]:
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : None,
}
solv_ansatz01 = SolveCircuit(circ_ansatz01, **class_dict)
solv_ansatz01.run()

In [ ]:
solv_ansatz01.state

### 1.2 Parent Hamiltonian General ansatz

The function *ansatz_qlm_02* implements a generalization of the *ansatz_qlm_01* one. In the *ansatz_qlm_01* all the qubits has the same operations with the same parameters. In the *ansatz_qlm_02* each qubit has the same operations but each operation will have a different  parameter.

The *SolveCircuit* class can be used for solving the ansatz

In [ ]:
from PH.ansatzes import ansatz_qlm_02

In [ ]:
n_qubits = 12
depth = 3
circ_ansatz02 = ansatz_qlm_02(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz02 --svg

In [ ]:
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : None,
}
solv_ansatz02 = SolveCircuit(circ_ansatz02, **class_dict)
solv_ansatz02.run()

In [ ]:
solv_ansatz02.state

### 1.3 Other ansatzes

For using the **PH** library for computing Parent Hamiltonians the mandatory input will be the list of amplitudes. We can use different ansatzes that are avialable from *Atos myqlm* library.

In [ ]:
from qat.fermion.circuits import make_ldca_circ, make_general_hwe_circ

In [ ]:
nqubit = 8
depth = 3
lda_circ = make_ldca_circ(nqubit, depth)

In [ ]:
%qatdisplay lda_circ --svg

In [ ]:
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : None,
}
solv_lda = SolveCircuit(lda_circ, **class_dict)
solv_lda.run()
solv_lda.state

In [ ]:
hwe_circ = make_general_hwe_circ(nqubit, n_cycles=3)

In [ ]:
%qatdisplay hwe_circ --svg

In [ ]:
class_dict = {
    "qpu" : qpu_mps,
    "parameters" : None,
}
solv_hwe = SolveCircuit(hwe_circ, **class_dict)
solv_hwe.run()
solv_hwe.state